In [1]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2
# for visualization self-attention
!pip install -q bertviz~=1.4.0

### The Encoder

Transformer 由 N 個 Encoder 組成，每個 Encoder 將其輸出送到下一個 Encoder。最終的 Encoder 返回 **輸入** 的 **向量** 表示。為了說明，從現在開始，我們將使用 N=2 的值。

![encoder in a nutshell](https://www.alexisalulema.com/wp-content/uploads/2022/08/encoders.png)

每個解碼器區塊由兩個子層組成：

1. 多頭注意力 (Multi-head attention)
2. 前饋神經網路 (Feedforward Network)

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/encoders.inside-768x315.png)

在開始解釋這兩個組件之前，有必要先了解自注意力 (self-attention) 機制。

#### Self-Attention

考慮以下句子：

```
John and Paul wrote several songs when they were inspired.
```

在這個句子中，自注意力機制計算每個詞的表示，並且與句子中其他詞的關係提供了更多關於該詞的信息。例如，「they」這個詞應該與「John」和「Paul」相關，而不是與「songs」相關。

讓我們簡單以一個視覺化的方式來解釋自注意力機制：




In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from pprint import pprint

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

text = "John and Paul wrote several songs when they were inspired."
# Tokenize input text
text_to_token_id = tokenizer(
    text,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

model = AutoModel.from_pretrained(
    model_name,
    output_attentions=True, # Whether the model returns attentions weights
    )

# Run the text through BERT
with torch.no_grad():
    outputs = model(**text_to_token_id)
    # Retrieve attention from model outputs
    attention = outputs.attentions

tokens = tokenizer.convert_ids_to_tokens(text_to_token_id['input_ids'][0])  # Convert input ids to token strings
pprint(tokens)


In [ ]:
from bertviz import (
  model_view,
  head_view,
)

# Visualize the self-attention of the model
# Review the relationship of term 'they' along with 'John' and 'Paul'
head_view(
  attention,
  tokens,
  layer=8,
  heads=[9],
  )

以一個更簡單的例子「The sky is blue」來理解自注意力機制如何運作，編碼器 (Encoder) 接收句子中每個詞的維度為 $\ d_{model} = 512 $ 的 Word Embedding 向量，例如：

$\ x_1 = [3.23, 0.65, ..., 4.78] \Rightarrow \text{"The"} $

$\ x_2 = [1.26, 6.35, ..., 7.99] \Rightarrow \text{"sky"} $

$\ x_3 = [9.25, 1.68, ..., 4.26] \Rightarrow \text{"is"} $

$\ x_4 = [6.84, 2.98, ..., 11.48] \Rightarrow \text{"blue"} $

有了這些向量，我們可以組裝 Embedding 矩陣 $\ X $，其維度為$\ d = [4 \times 512] $：

![](https://alexisalulema.com/wp-content/uploads/2022/08/embedding_matrix.gif)

我們將從這個矩陣 $\ X $ 中創建三個額外的矩陣，作為「自注意力機制」的一部分：

* $\ Q $, query matrix
* $\ K $, key matrix
* $\ V $, value matrix

要創建這些陣列，我們還需要三個新的權重矩陣 (weight matrices)。

原始論文中使用的維度是 $\ d_k = 64 $；因此，權重向量的維度將是 $\ d_{model} \times d_k \Rightarrow 512 \times 64 $，這些矩陣會用隨機值初始化：

* $\ W^Q $, query weight matrix
* $\ W^K $, key weight matrix
* $\ W^V $, value weight matrix

權重矩陣攜帶了在訓練過程中學到的最佳值，因此每個矩陣 $\ Q $、$\ K $ 和 $\ V $ 是 Embedding 矩陣 $\ X $ 與相應權重矩陣的乘積，這會生成 $\ 4 \times 64 $ 的矩陣：

* $\ Q = X × W^Q $
* $\ K = X × W^K $
* $\ V = X × W^V $ 

![](https://alexisalulema.com/wp-content/uploads/2022/08/QKV.gif)

每個矩陣中的四行分別代表句子「The sky is blue」中的每個詞。


##### Self-attention Mechanism Process

1. 計算點積 (dot product) $\ Q \cdot K^T $

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/QdotKT-1-1024x140.gif)

結果陣列的元素表示詞語之間的關係。例如，$\ q_1 \cdot k_1 $ 是詞「The」與其自身的關係，$\ q_1 \cdot k_3 $ 是「The」與「is」之間的關係。「sky」與「blue」之間的關係 $\ q_2 \cdot k_4 $ 會有稍高的值，因為名詞和形容詞之間存在關係。例如：

![](https://alexisalulema.com/wp-content/uploads/2022/08/QdotKT.2.gif)

這樣，我們可以說計算查詢矩陣 $\ Q $ 和鍵矩陣 $\ K^T $ 之間的點積，基本上給出了相似度值，這有助於我們理解句子中每個詞與所有其他詞的相似程度。

2. 計算 $\ QK^T / \sqrt{d_k} $。這個操作有助於獲得穩定的梯度，其中 $\ d_k = 64 $ 是鍵向量的維度。

![](https://alexisalulema.com/wp-content/uploads/2022/08/QdotKTSqrtDK.gif)

結果矩陣的值必須正規化，如果我們使用函數 $\ \text{Softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) $，我們可以將值轉化為 0 到 1 的範圍。

每行的值之和等於 1。通過這些值，我們可以理解句子中每個詞與所有其他詞的關係。這稱為分數矩陣 (score matrix)。

3. 接下來，我們需要計算注意力矩陣 (attention matrix) $\ Z $：

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/SoftmaxV.gif)

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/Z.SoftmaxV-1024x148.gif)

注意力矩陣 $\ Z $ 是一個具有 4 行和 512 列的矩陣，對於示例句子來說。每行對應於相應詞的自注意力 (self-attention) 向量。

自注意力 (self-attention) 機制被稱為縮放點積注意力 (scaled dot product attention)，因為我們在計算向量 $\ Q $ 和 $\ K $ 之間的點積並通過 $\ \sqrt{d_k} $ 來縮放值。

我們也可以透過視覺化的方式來理解自注意力機制中 $\ Ｑ $ 和 $\ Ｋ $ 的作用：


In [ ]:
from bertviz.transformers_neuron_view import BertModel, BertTokenizer
from bertviz.neuron_view import show

tokenizer = BertTokenizer.from_pretrained(
  'bert-base-uncased',
  do_lower_case=True)
model = BertModel.from_pretrained(
  'bert-base-uncased',
  output_attentions=True)
show(
  model=model,
  model_type='bert',
  tokenizer=tokenizer,
  sentence_a=text,
  layer=8,
  head=9,
  )

#### Multi-head attention

對於 Transformer，我們將計算多個注意力 (multiple attention) 矩陣。但為什麼我們需要多個陣列呢？這有助於在語境中詞義模糊的情況下，例如：

```
Tom was crying because he was blue.
```

單一的注意力機制可能會決定 Tom 哭泣是因為他的顏色是藍色，這是由於「Tom」這個詞的影響。如果大多數句子中「blue」表示顏色，只有一個「注意力頭 (attention head)」的機制將正確地學習到它是一種顏色。然而，擁有「多個注意力頭 (multiple attention heads)」，其中一個注意力機制更有可能從句子中學習到「blue」表示心情，通過串聯「多個注意力頭 (multiple attention heads)」的結果，注意力矩陣將更加準確。

我們如何計算多個注意力矩陣？假設我們要計算兩個注意力矩陣：$\ Z_1 $ 和 $\ Z_2 $。

要計算 $\ Z_1 $，首先，我們創建三個矩陣 $\ Q_1 $、$\ K_1 $ 和 $\ V_1 $，這意味著將 Embedding 矩陣與三個權重矩陣 $\ W_1^Q $、$\ W_1^K $ 和 $\ W_1^V $ 相乘。現在，注意力矩陣的計算方式如下：

$\ Z_1 = \text{Softmax}\left(\frac{Q_1K_1^T}{\sqrt{d_k}} \cdot V_1 \right) $

同樣地，對於 $\ Z_2 $ 也是如此。

$\ Z_2 = \text{Softmax}\left(\frac{Q_2K_2^T}{\sqrt{d_k}} \cdot V_2 \right) $

這樣，我們可以計算任意數量的注意力矩陣。假設我們需要八個注意力矩陣，在《[Attention is all you need](https://arxiv.org/abs/1706.03762?context=cs)》中提到的數值。在這種情況下，我們可以將所有的注意力頭拼接 (concatenate) 起來，並將結果乘以一個新的權重矩陣 $\ W_0 $，該矩陣經過訓練以表示注意力機制的最佳值。

Multi-head attention $\ = Concatenate( Z_1, Z_2, Z_3, Z_4, Z_5, Z_6, Z_7, Z_8 ) \cdot W_0 $

#### Feedforward Network

Feedforward Network 由兩個帶有 ReLU 激活的全連接層組成。

另一個用於連接輸入和編碼器 (Encoder) 的組件是 **Add and norm** 組件，這是一種連接，隨後進行 **layer normalization**。

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/addNnorm.png)

通過 Layer normalization 防止每層中的值發生劇烈變化來實現更快的訓練。

